In [32]:
from policyengine import Simulation
import pandas as pd
import numpy as np
from microdf import MicroDataFrame

START_YEAR = 2026
COUNT_YEARS = 5
FORECAST_YEARS = list(range(START_YEAR, START_YEAR + COUNT_YEARS))

GROWFACTORS = {
    "earned_income": {
        2026: 0.02,
        2027: 0.02,
        2028: 0.02,
        2029: 0.02,
        2030: 0.02,
    },
    "mixed_income": {
        2026: 0.02,
        2027: 0.02,
        2028: 0.02,
        2029: 0.02,
        2030: 0.02,
    },
    "capital_income": {
        2026: 0.02,
        2027: 0.02,
        2028: 0.02,
        2029: 0.02,
        2030: 0.02,
    },
    "inflation": {
        2026: 0.02,
        2027: 0.02,
        2028: 0.02,
        2029: 0.02,
        2030: 0.02,
    },
}


def get_cumulative_growth(base_year: int, target_year: int, growth_rates: dict) -> float:
    cumulative_growth = 1
    for year in range(base_year + 1, target_year + 1):
        cumulative_growth *= (1 + growth_rates[year])
    return cumulative_growth

subsample = None
growfactors = GROWFACTORS

simulation = Simulation(
    country="uk",
    scope="macro",
    subsample=subsample,
).baseline_simulation

for year in FORECAST_YEARS:
    # Update household data
    cumulative_earnings_growth = get_cumulative_growth(START_YEAR, year, growfactors["earned_income"])
    print(f"year: {year}, cumulative_earnings_growth: {cumulative_earnings_growth}")
    simulation.set_input(
        "employment_income",
        year,
        simulation.calculate("employment_income", START_YEAR) * get_cumulative_growth(START_YEAR, year, growfactors["earned_income"]),
    )
    simulation.set_input(
        "self_employment_income",
        year,
        simulation.calculate("self_employment_income", START_YEAR) * get_cumulative_growth(START_YEAR, year, growfactors["mixed_income"]),
    )
    simulation.set_input(
        "dividend_income",
        year,
        simulation.calculate("dividend_income", START_YEAR) * get_cumulative_growth(START_YEAR, year, growfactors["capital_income"]),
    )
    simulation.set_input(
        "consumption",
        year,
        simulation.calculate("consumption", START_YEAR) * get_cumulative_growth(START_YEAR, year, growfactors["inflation"]),
    )

    # Update government policy parameters
    cpi_parameter = simulation.tax_benefit_system.parameters.gov.obr.consumer_price_index
    cpi_parameter.update(
        period=f"year:{year}:1",
        value=cpi_parameter(2025) * get_cumulative_growth(2025, year, growfactors["inflation"]),
    )


df = pd.DataFrame()
for year in range(2025, START_YEAR + COUNT_YEARS):
    year_df = simulation.calculate_dataframe([
        "household_id",
        "household_weight",
        "household_count_people",
        "household_income_decile",
        "household_net_income",
        "employment_income",
        "self_employment_income",
        "dividend_income",
        "consumption",
        "in_poverty",
    ], period=year).reset_index()
    year_df["year"] = year
    df = pd.concat([
        df,
        year_df,
    ])



year: 2026, cumulative_earnings_growth: 1
year: 2027, cumulative_earnings_growth: 1.02
year: 2028, cumulative_earnings_growth: 1.0404
year: 2029, cumulative_earnings_growth: 1.061208
year: 2030, cumulative_earnings_growth: 1.08243216


In [33]:
from microdf import MicroDataFrame

df = MicroDataFrame(df, weights="household_weight")

In [34]:
df.groupby("year").household_net_income.sum()

year
2025    1.589477e+12
2026    1.642204e+12
2027    1.688124e+12
2028    1.732616e+12
2029    1.776034e+12
2030    1.805807e+12
dtype: float64